# ***Precio Promedio por M2 cuadrados***

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "tulancingo" in nombre 
]
# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_tulancingo',
 'df_alfa_agosto_2024_tulancingo',
 'df_alfa_febrero_2024_tulancingo',
 'df_alfa_julio_2024_tulancingo',
 'df_alfa_junio_2024_tulancingo',
 'df_alfa_marzo_2024_tulancingo',
 'df_alfa_mayo_2024_tulancingo',
 'df_alfa_q_jul_2022_tulancingo',
 'df_alfa_q_jul_2023_tulancingo',
 'df_alfa_q_jun_2023_tulancingo',
 'df_alfa_q_mar_2023_tulancingo',
 'df_alfa_q_may_2022_tulancingo',
 'df_alfa_q_may_2023_tulancingo',
 'df_alfa_q_oct_2022_tulancingo',
 'df_alfa_q_oct_2023_tulancingo',
 'df_alfa_q_sep_2023_tulancingo']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Precio':'precio','metros_total':'m2_total','precio_m2_cons':'precio_m2_construido','metros_construido':'m2_construido','precio_m2_contruido':'precio_m2_construido','id_jul_23':'id','m2_contruido': 'm2_construido','m_construido': 'm2_construido','Metros_construido':'m2_construido','segmento':'categoria','Category':'categoria','m_total':'m2_total','m_construido':'m2_construido'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['precio', 'm2_total','m2_construido', 'precio_m2_construido', 'precio_m2_terreno']]
    # Añadir el DataFrame a la listaprecio_m2_construido
    dataframes_list.append(segment_df)

### *DF CON RANGOS DE M2 Y SU PRECIO*

In [4]:
new_dataframes_list = []

# Definir los límites de los rangos
rangos = [0, 100, 150, 200, 250, 300, float('inf')]
labels = ['[0-100]', '[100-150]', '[150-200]', '[200-250]', '[250-300]', '[> 300]']

# Iterar sobre cada DataFrame en la lista original
for df in dataframes_list:
    # Crear una copia del DataFrame
    modified_df = df.copy()
    # Asignar el rango correspondiente a cada valor de 'm2_construido'
    modified_df['rango'] = pd.cut(modified_df['m2_total'], bins=rangos, labels=labels, right=False)
    # Crear un nuevo DataFrame solo con las columnas 'rango' y 'precio_m2_terreno'
    new_df = modified_df[['rango', 'precio_m2_terreno']].copy()
    # Agregar el nuevo DataFrame a la lista
    new_dataframes_list.append(new_df)

# Imprimir los primeros registros de cada nuevo DataFrame
for idx, df in enumerate(new_dataframes_list):
    print(f"Primeros registros de nuevo DataFrame {idx + 1}:")
    print(df.head())
    print()  # Agregar una línea en blanco entre cada DataFrame


Primeros registros de nuevo DataFrame 1:
       rango  precio_m2_terreno
0    [> 300]           16597.51
1  [200-250]           21096.49
2  [200-250]           22500.00
3  [150-200]           26875.00
4  [150-200]           26309.77

Primeros registros de nuevo DataFrame 2:
       rango  precio_m2_terreno
0  [100-150]           23800.00
1  [150-200]            6875.00
2  [100-150]           20168.07
3  [100-150]            3869.57
4  [100-150]            4333.33

Primeros registros de nuevo DataFrame 3:
       rango  precio_m2_terreno
0    [0-100]            3977.47
1    [0-100]            9716.98
2    [0-100]           11304.35
3  [150-200]            3247.19
4  [150-200]            3247.19

Primeros registros de nuevo DataFrame 4:
       rango  precio_m2_terreno
0  [150-200]            2866.67
1  [150-200]            2866.67
2  [150-200]            2866.67
3  [250-300]            1535.71
4  [150-200]            2866.67

Primeros registros de nuevo DataFrame 5:
       rango  precio_m2

### *Promedio de precio_m2_terreno por rango CADA DF* 

In [5]:
# Iterar sobre cada DataFrame modificado en new_dataframes_list
for idx, df in enumerate(new_dataframes_list):
    print(f"Promedio de precio_m2_terreno por rango en el DataFrame {idx + 1}:")
    # Agrupar el DataFrame por la columna 'rango' y calcular el promedio de 'precio_m2_terreno' para cada grupo
    promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()
    print(promedios_por_rango)
    print()  # Agregar una línea en blanco entre cada resultado

Promedio de precio_m2_terreno por rango en el DataFrame 1:
rango
[0-100]      12465.619487
[100-150]     9887.987609
[150-200]     8678.312500
[200-250]    11988.958125
[250-300]     5109.025000
[> 300]       6958.146000
Name: precio_m2_terreno, dtype: float64

Promedio de precio_m2_terreno por rango en el DataFrame 2:
rango
[0-100]      15376.508571
[100-150]     8244.480189
[150-200]     6329.609474
[200-250]    11840.073182
[250-300]     3705.227500
[> 300]       3477.185714
Name: precio_m2_terreno, dtype: float64

Promedio de precio_m2_terreno por rango en el DataFrame 3:
rango
[0-100]      12507.756389
[100-150]     9504.902245
[150-200]     9325.852703
[200-250]    11324.768500
[250-300]    14701.755000
[> 300]       6651.282500
Name: precio_m2_terreno, dtype: float64

Promedio de precio_m2_terreno por rango en el DataFrame 4:
rango
[0-100]      14603.811176
[100-150]     8423.657759
[150-200]     6372.095385
[200-250]    11910.425455
[250-300]     2708.857778
[> 300]       8333.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14128\2294499642.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()


In [6]:
# Inicializar un diccionario para almacenar los promedios de precio por metro cuadrado para cada rango
promedios_por_rango_dict = {}
# Iterar sobre cada DataFrame modificado en new_dataframes_list
for idx, df in enumerate(new_dataframes_list):
    # Agrupar el DataFrame por la columna 'rango' y calcular el promedio de 'precio_m2_terreno' para cada grupo
    promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()
    # Actualizar el diccionario con los promedios calculados
    promedios_por_rango_dict[f'DataFrame_{idx + 1}'] = promedios_por_rango

# Convertir el diccionario en un DataFrame
rango_precio_m2 = pd.DataFrame(promedios_por_rango_dict)
print("DataFrame 'rango_precio_m2':")
rango_precio_m2

DataFrame 'rango_precio_m2':


C:\Users\yoe11\AppData\Local\Temp\ipykernel_14128\225855980.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  promedios_por_rango = df.groupby('rango')['precio_m2_terreno'].mean()


,DataFrame_1,DataFrame_2,DataFrame_3,DataFrame_4,DataFrame_5,DataFrame_6,DataFrame_7,DataFrame_8,DataFrame_9,DataFrame_10,DataFrame_11,DataFrame_12,DataFrame_13,DataFrame_14,DataFrame_15,DataFrame_16
rango,,,,,,,,,,,,,,,,
[0-100],12465.619487,15376.508571,12507.756389,14603.811176,11933.070000,11681.987619,10643.768235,11559.894167,12174.243333,12401.016757,11730.660000,10136.975000,12028.848333,12252.039412,11408.553529,11725.755000
[100-150],9887.987609,8244.480189,9504.902245,8423.657759,10239.861351,10239.861351,8464.221094,11262.859302,13373.047391,13492.396230,9667.866167,11910.752174,10681.750000,12978.701311,13353.897500,13290.044839
[150-200],8678.312500,6329.609474,9325.852703,6372.095385,8810.481786,8810.481786,6335.340811,10694.441667,11109.317500,22377.726000,13442.417273,10694.441667,11763.568182,19621.355882,15188.125000,17144.500000
[200-250],11988.958125,11840.073182,11324.768500,11910.425455,11425.256471,11425.256471,9949.946250,12101.732000,8605.074615,24936.829655,11601.462632,12456.120000,11214.658824,15771.517241,14068.180000,14068.180000
[250-300],5109.025000,3705.227500,14701.755000,2708.857778,3322.367500,3322.367500,2614.099000,5719.360000,7115.286667,22848.477143,4520.060000,9351.616000,9692.476667,10043.803333,21403.510000,21403.510000
[> 300],6958.146000,3477.185714,6651.282500,8333.140000,7805.924167,7805.924167,5522.297500,4171.867500,8589.407000,9625.699167,7322.458333,9009.301000,8031.125714,14272.792500,7107.140000,7107.140000


In [7]:
# Calcular el promedio general de cada rango en todas las columnas de rango_precio_m2
promedio_general_por_rango = rango_precio_m2.mean(axis=1)
# Crear un nuevo DataFrame 'Precio_mean_M2' con los rangos y sus promedios generales
Precio_mean_M2 = pd.DataFrame({'Rango': promedio_general_por_rango.index, 'Precio_m2_mean': promedio_general_por_rango.values})

print("DataFrame 'Precio_mean_M2':")
Precio_mean_M2

DataFrame 'Precio_mean_M2':


,Rango,Precio_m2_mean
0,[0-100],12164.406688
1,[100-150],10938.517907
2,[150-200],11668.629226
3,[200-250],12793.027464
4,[250-300],9223.862443
5,[> 300],7611.926954


# Gráfica

In [8]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9']
porcentajes = Precio_mean_M2['Precio_m2_mean']

# Convertir la columna 'Precio_m2_mean' a números de punto flotante
#Precio_mean_M2['Precio_m2_mean'] = Precio_mean_M2['Precio_m2_mean'].str.replace('$', '').str.replace(',', '').astype(float)
# Convertir los valores a cadenas con el formato deseado
textos_formateados = [f'${round(valor, 2):,.2f}' for valor in Precio_mean_M2['Precio_m2_mean']]

fig = go.Figure()
# Obtener el valor máximo redondeado al próximo múltiplo de 5000
max_valor = int(Precio_mean_M2['Precio_m2_mean'].max() / 5000 + 1) * 5000

# Agregar los datos como barras
fig.add_trace(go.Bar(
    x=Precio_mean_M2['Rango'],  # Eje x: Modalidades
    y=Precio_mean_M2['Precio_m2_mean'],  # Eje y: Porcentajes
    marker_color=colores,  # Especifica los colores de las barras
    text=textos_formateados,  # Texto que se mostrará en las barras (porcentaje)
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))

# Personalizar el diseño de la gráfica
fig.update_layout(
    #title='Precio promedio por M2 totales.',  # Título de la gráfica
    #xaxis=dict(title='Rango'),  # Título del eje x
    yaxis=dict(
        #title='Precio', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
        tickvals=list(range(0, max_valor + 5000, 5000)),  # Valores de los ticks del eje y
        ticktext=[f'${valor:.2f}' for valor in range(0, max_valor + 5000, 5000)]  # Textos de los ticks del eje y
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    height=500,  # Altura de la gráfica
    margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)

)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')
guardar_grafico_como_html(fig, 'g_bar_PrecioPromedio_m2', carpeta='assets/graficas')
fig.show()

## POR M2_CONSTRUCCIÓN

In [9]:
new_dataframes_list = []

# Definir los límites de los rangos
rangos = [0, 100, 150, 200, 250, 300, float('inf')]
labels = ['[0-100]', '[100-150]', '[150-200]', '[200-250]', '[250-300]', '[> 300]']

# Iterar sobre cada DataFrame en la lista original
for df in dataframes_list:
    # Crear una copia del DataFrame
    modified_df = df.copy()
    # Asignar el rango correspondiente a cada valor de 'm2_construido'
    modified_df['rango'] = pd.cut(modified_df['m2_total'], bins=rangos, labels=labels, right=False)
    # Crear un nuevo DataFrame solo con las columnas 'rango' y 'precio_m2_terreno'
    new_df = modified_df[['rango', 'precio_m2_construido']].copy()
    # Agregar el nuevo DataFrame a la lista
    new_dataframes_list.append(new_df)

# Imprimir los primeros registros de cada nuevo DataFrame
for idx, df in enumerate(new_dataframes_list):
    print(f"Primeros registros de nuevo DataFrame {idx + 1}:")
    print(df.head())
    print()  # Agregar una línea en blanco entre cada DataFrame


Primeros registros de nuevo DataFrame 1:
       rango  precio_m2_construido
0    [> 300]              17391.30
1  [200-250]              13361.11
2  [200-250]              15679.44
3  [150-200]              17916.67
4  [150-200]              26309.77

Primeros registros de nuevo DataFrame 2:
       rango  precio_m2_construido
0  [100-150]              11359.09
1  [150-200]              11000.00
2  [100-150]              15584.42
3  [100-150]               2225.00
4  [100-150]               2275.00

Primeros registros de nuevo DataFrame 3:
       rango  precio_m2_construido
0    [0-100]               3977.47
1    [0-100]               9716.98
2    [0-100]              11304.35
3  [150-200]               3247.19
4  [150-200]               3247.19

Primeros registros de nuevo DataFrame 4:
       rango  precio_m2_construido
0  [150-200]               1535.71
1  [150-200]               1535.71
2  [150-200]               1535.71
3  [250-300]               2866.67
4  [150-200]               1

### *Promedio de precio_m2_terreno por rango CADA DF* 

In [10]:
# Iterar sobre cada DataFrame modificado en new_dataframes_list
for idx, df in enumerate(new_dataframes_list):
    print(f"Promedio de precio_m2_construido por rango en el DataFrame {idx + 1}:")
    # Agrupar el DataFrame por la columna 'rango' y calcular el promedio de 'precio_m2_terreno' para cada grupo
    promedios_por_rango = df.groupby('rango')['precio_m2_construido'].mean()
    print(promedios_por_rango)
    print()  # Agregar una línea en blanco entre cada resultado

Promedio de precio_m2_construido por rango en el DataFrame 1:
rango
[0-100]      12132.946410
[100-150]     9493.749130
[150-200]     7602.253333
[200-250]    11079.687500
[250-300]     6623.560000
[> 300]       7882.844000
Name: precio_m2_construido, dtype: float64

Promedio de precio_m2_construido por rango en el DataFrame 2:
rango
[0-100]      13398.620476
[100-150]     8023.152075
[150-200]     5591.370526
[200-250]     9965.436364
[250-300]     4158.140000
[> 300]       7423.210000
Name: precio_m2_construido, dtype: float64

Promedio de precio_m2_construido por rango en el DataFrame 3:
rango
[0-100]      12417.566944
[100-150]     8784.592857
[150-200]     7339.657297
[200-250]     9159.341500
[250-300]    12975.380000
[> 300]      10851.612500
Name: precio_m2_construido, dtype: float64

Promedio de precio_m2_construido por rango en el DataFrame 4:
rango
[0-100]      11231.851176
[100-150]     7835.577414
[150-200]     5251.402308
[200-250]     9752.788182
[250-300]     2932.50111

C:\Users\yoe11\AppData\Local\Temp\ipykernel_14128\2302470058.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [11]:
# Inicializar un diccionario para almacenar los promedios de precio por metro cuadrado para cada rango
promedios_por_rango_dict = {}
# Iterar sobre cada DataFrame modificado en new_dataframes_list
for idx, df in enumerate(new_dataframes_list):
    # Agrupar el DataFrame por la columna 'rango' y calcular el promedio de 'precio_m2_terreno' para cada grupo
    promedios_por_rango = df.groupby('rango')['precio_m2_construido'].mean()
    # Actualizar el diccionario con los promedios calculados
    promedios_por_rango_dict[f'DataFrame_{idx + 1}'] = promedios_por_rango

# Convertir el diccionario en un DataFrame
rango_precio_m2 = pd.DataFrame(promedios_por_rango_dict)
print("DataFrame 'rango_precio_m2':")
rango_precio_m2

DataFrame 'rango_precio_m2':


C:\Users\yoe11\AppData\Local\Temp\ipykernel_14128\905179478.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,DataFrame_1,DataFrame_2,DataFrame_3,DataFrame_4,DataFrame_5,DataFrame_6,DataFrame_7,DataFrame_8,DataFrame_9,DataFrame_10,DataFrame_11,DataFrame_12,DataFrame_13,DataFrame_14,DataFrame_15,DataFrame_16
rango,,,,,,,,,,,,,,,,
[0-100],12132.946410,13398.620476,12417.566944,11231.851176,11576.064286,11324.981905,9833.762353,10856.348333,9464.051212,44951.838378,11177.843810,10792.037000,13865.805333,11491.667059,10532.587647,10399.207500
[100-150],9493.749130,8023.152075,8784.592857,7835.577414,9989.983514,9989.983514,7425.449688,10437.698837,12929.066522,18626.723607,9450.451167,11121.045217,10219.417826,11297.298361,12443.414375,12389.240323
[150-200],7602.253333,5591.370526,7339.657297,5251.402308,6620.806786,6620.806786,5255.186757,10662.036667,16019.593750,74426.180800,10751.630909,10662.036667,12328.280000,17001.294118,14826.350000,14464.696000
[200-250],11079.687500,9965.436364,9159.341500,9752.788182,10354.961765,10354.961765,9592.938750,14061.564000,10420.107692,21346.302414,9186.240526,13268.088182,13788.504706,14446.148621,19144.960000,19144.960000
[250-300],6623.560000,4158.140000,12975.380000,2932.501111,4745.115000,4745.115000,2953.353000,11826.490000,10840.203333,19597.551429,8434.551429,11407.004000,7485.908333,8224.526667,14186.050000,14186.050000
[> 300],7882.844000,7423.210000,10851.612500,12134.550000,14087.778333,14087.778333,8307.410000,6889.322500,12510.741000,16930.669167,9533.111667,13879.614000,5469.620000,15953.347500,12587.090000,12587.090000


In [12]:
# Calcular el promedio general de cada rango en todas las columnas de rango_precio_m2
promedio_general_por_rango = rango_precio_m2.mean(axis=1)
# Crear un nuevo DataFrame 'Precio_mean_M2' con los rangos y sus promedios generales
Precio_mean_M2 = pd.DataFrame({'Rango': promedio_general_por_rango.index, 'Precio_m2_mean': promedio_general_por_rango.values})

print("DataFrame 'Precio_mean_M2':")
Precio_mean_M2

DataFrame 'Precio_mean_M2':


,Rango,Precio_m2_mean
0,[0-100],13465.448739
1,[100-150],10653.552777
2,[150-200],14088.973919
3,[200-250],12816.686998
4,[250-300],9082.593706
5,[> 300],11319.736812


In [13]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9']
porcentajes = Precio_mean_M2['Precio_m2_mean']

# Convertir la columna 'Precio_m2_mean' a números de punto flotante
#Precio_mean_M2['Precio_m2_mean'] = Precio_mean_M2['Precio_m2_mean'].str.replace('$', '').str.replace(',', '').astype(float)
# Convertir los valores a cadenas con el formato deseado
textos_formateados = [f'${round(valor, 2):,.2f}' for valor in Precio_mean_M2['Precio_m2_mean']]

fig = go.Figure()
# Obtener el valor máximo redondeado al próximo múltiplo de 5000
max_valor = int(Precio_mean_M2['Precio_m2_mean'].max() / 5000 + 1) * 5000

# Agregar los datos como barras
fig.add_trace(go.Bar(
    x=Precio_mean_M2['Rango'],  # Eje x: Modalidades
    y=Precio_mean_M2['Precio_m2_mean'],  # Eje y: Porcentajes
    marker_color=colores,  # Especifica los colores de las barras
    text=textos_formateados,  # Texto que se mostrará en las barras (porcentaje)
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))

# Personalizar el diseño de la gráfica
fig.update_layout(
    #title='Precio promedio por M2 totales.',  # Título de la gráfica
    xaxis=dict(title='M² Construidos'),  # Título del eje x
    yaxis=dict(
        title='Precio promedio', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
        tickvals=list(range(0, max_valor + 5000, 5000)),  # Valores de los ticks del eje y
        ticktext=[f'${valor:.2f}' for valor in range(0, max_valor + 5000, 5000)]  # Textos de los ticks del eje y
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    height=500,  # Altura de la gráfica
    margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)

)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)


guardar_grafico_como_html(fig, 'g_bar_PrecioPromedio_m2', carpeta='assets/graficas')
fig.show()
